In [ ]:
import os
import sys

sys.path.append(os.path.join(os.getcwd(), 'Mrlishu'))
if os.path.exists('/content'):
    os.chdir('/content')
    CODE_DIR = 'PHM-Tongji'
    if not os.path.exists(CODE_DIR):
        !git clone https://github.com/MrLishu/PHM-Tongji.git $CODE_DIR
    os.chdir(f'./{CODE_DIR}')

    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

In [ ]:
len = {}
for i in (1, 4, 6):
  len[f'c{i}'] = []
  for j in range(315):
    data = pd.read_csv(rf'/content/drive/MyDrive/PHM-Tongji/data/raw/PHM2010/c{i}/c_{i}_{j + 1:03d}.csv')
    len[f'c{i}'].append(data.shape[0])

In [ ]:
import numpy as np
import pandas as pd
import pywt


class PHMDataset(Dataset):
    def __init__(self, data_path, dataset_name, train=True, data_num=315, usecols=('Ax', 'Ay', 'Az'), resample_number=1024, step=1, totalscale=256, wavename='morl'):
        self.data_path = data_path
        self.dataset_name = dataset_name
        self.train = train
        self.data_num = data_num
        self.usecols = usecols
        self.resample_number = resample_number
        self.step = step

        self.signal_length = pd.read_csv(os.path.join(data_path, r'processed/PHM2010/signal_length.csv'))
        self.labelset = {}
        for name in dataset_name:
          self.labelset[name] = pd.read_csv(os.path.join(data_path, rf'raw/PHM2010/{name}_wear.csv'))
        self.columns = ['Fx', 'Fy', 'Fz', 'Ax', 'Ay', 'Az', 'Null']
        sampling_rate = 50000

        fc = pywt.central_frequency(wavename)
        cparam = 2 * fc * totalscale
        self.scales = cparam / np.arange(totalscale, 0, -1)
        self.wavename = wavename
        self.resample_rate = sampling_rate / step
      
    def __len__(self):
        return self.data_num
    
    def __getitem__(self, idx):
        name = self.dataset_name[idx // self.data_num]
        ci = name[1]
        i = idx % self.data_num
        len = self.signal_length[name][i]

        if self.train:
            start = np.random.randint(len - self.resample_number * self.step)
        else:
            start = len - self.resample_number * self.step // 2
        signal = pd.read_csv(os.path.join(self.data_path, rf'raw/PHM2010/{name}/c_{ci}_{i + 1:03d}.csv'), names=self.columns, usecols=self.usecols, skiprows=start, nrows=self.step * self.resample_number)
        resampled = signal.to_numpy().T[:, ::self.step]

        resampled -= resampled.mean(axis=-1)[:, np.newaxis]
        resampled /= resampled.std(axis=-1)[:, np.newaxis]

        cwtmatr, frequencies = pywt.cwt(resampled, self.scales, self.wavename, 1 / self.resample_rate)
        cwtmatr = cwtmatr.transpose(1, 0, 2)

        label = self.labelset[name].iloc[i, 1:].mean()
        return cwtmatr, label

In [ ]:
train_dataset = PHMDataset(r'/content/drive/MyDrive/PHM-Tongji/data', ['c1', 'c4'])
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

In [ ]:
data, label = next(iter(train_dataloader))
data.shape, label.shape

In [ ]:
model = models.resnet18(pretrained=False)
# model.conv1 = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
model.fc = nn.Linear(in_features=512, out_features=1, bias=True)

In [ ]:
device = device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)
scheduler = None
num_epochs = 25

In [ ]:
loss_table = []

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    running_loss = 0.0

    # Iterate over data.
    for step, (inputs, labels) in enumerate(train_dataloader):
        model.train()
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels[:, np.newaxis])

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
      
    loss_table.append(running_loss)
    print(f'Running loss: {running_loss:.3f}')